# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"


## Generate Cities List

In [9]:
import numpy as np
#Define our lat/lon range and increments between the range
min_lat = 32
max_lat = 42
min_lon = -114
max_lon = -125
increment = 1

#Build our lat/lon list within range
lat_lons = []

lats = [round(num, 2) for num in np.arange(min_lat, max_lat, increment).tolist()]
lons = [round(num, 2) for num in np.arange(min_lon, max_lon, -(increment)).tolist()]
lat_lons = list(itertools.product(lats, lons))
len(lat_lons)

110

In [10]:
# List for holding lat_lngs and cities
cities = []

# Identify nearest city for each lat, lng combination
for lat_lon in lat_lons:
    city = citipy.nearest_city(lat_lon[0], lat_lon[1]).city_name
    print(city)
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

fortuna foothills
estacion coahuila
ensenada
primo tapia
rosarito
rancho palos verdes
port hueneme
isla vista
lompoc
lompoc
lompoc
fortuna foothills
algodones
imperial
poway
dana point
rancho palos verdes
port hueneme
lompoc
lompoc
lompoc
lompoc
lake havasu city
blythe
twentynine palms
yucaipa
hacienda heights
malibu
isla vista
lompoc
lompoc
morro bay
pacific grove
kingman
bullhead city
twentynine palms
barstow
california city
arvin
santa maria
pismo beach
morro bay
monterey
pacific grove
kingman
henderson
pahrump
ridgecrest
ridgecrest
porterville
avenal
king city
monterey
pacific grove
half moon bay
saint george
sunrise manor
pahrump
pahrump
exeter
orange cove
madera
los banos
santa cruz
half moon bay
pacifica
saint george
saint george
pahrump
pahrump
fallon
south lake tahoe
merced
garden acres
concord
petaluma
healdsburg
cedar city
west wendover
elko
fallon
fallon
fallon
south lake tahoe
north auburn
south yuba city
ukiah
ukiah
west wendover
elko
elko
winnemucca
fallon
fernley
sun va

In [ ]:
# Print the city count to confirm sufficient count
len(cities)


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
appid = weather_api_key

In [11]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
# http://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&cnt={cnt}&appid={YOUR_API_KEY}
# base_url = "http://history.openweathermap.org/data/2.5/history/city?"


counter = 1
city_data = []
cities =[]
for lat_lon in lat_lons:
    params = {
      "lat" : lat_lon[0],
      "lon" : lat_lon[1],
#       "type" : 'hour',
#       "start" : 1369728000,
#       "end" : 1369789200,
#         "q" : city,  
        "units" : "imperial",
        "appid" : weather_api_key
       }
    city = citipy.nearest_city(lat_lon[0], lat_lon[1]).city_name
    if city not in cities:
        cities.append(city)
    try:
        weather = requests.get(base_url, params=params).json()
        lat = weather["coord"]["lat"]
        lon = weather["coord"]["lon"]
        temp = weather["main"]["temp_max"]
        humid = weather["main"]["humidity"]
        cloud = weather["clouds"]["all"]
        wind = weather["wind"]["speed"]
        date = datetime.fromtimestamp(weather["dt"]-30).date()

        city_data.append({"City" : city
                        ,"Latitude" : lat
                        ,"Longitude" : lon
                        ,"Max Temperature": temp
                        ,"Humidity": humid
                        ,"Cloudiness": cloud
                        ,"Wind Speed" : wind
                        ,"Date" : date})

        print(f"Processing Record {counter} - {city}")
        counter = counter + 1
    except:
        print("City not found. Skipping")


Processing Record 1 - fortuna foothills
Processing Record 2 - estacion coahuila
Processing Record 3 - ensenada
Processing Record 4 - primo tapia
Processing Record 5 - rosarito
Processing Record 6 - rancho palos verdes
Processing Record 7 - port hueneme
Processing Record 8 - isla vista
Processing Record 9 - lompoc
Processing Record 10 - lompoc
Processing Record 11 - lompoc
Processing Record 12 - fortuna foothills
Processing Record 13 - algodones
Processing Record 14 - imperial
Processing Record 15 - poway
Processing Record 16 - dana point
Processing Record 17 - rancho palos verdes
Processing Record 18 - port hueneme
Processing Record 19 - lompoc
Processing Record 20 - lompoc
Processing Record 21 - lompoc
Processing Record 22 - lompoc
Processing Record 23 - lake havasu city
Processing Record 24 - blythe
Processing Record 25 - twentynine palms
Processing Record 26 - yucaipa
Processing Record 27 - hacienda heights
Processing Record 28 - malibu
Processing Record 29 - isla vista
Processing R

In [13]:
city_data_df = pd.DataFrame(city_data)
print(city_data_df)

                  City  Latitude  Longitude  Max Temperature  Humidity  \
0    fortuna foothills        32       -114            86.00        17   
1    estacion coahuila        32       -115            83.30        15   
2             ensenada        32       -116            66.00        89   
3          primo tapia        32       -117            66.00        82   
4             rosarito        32       -118            64.18        83   
..                 ...       ...        ...              ...       ...   
105         susanville        41       -120            77.00        17   
106         susanville        41       -121            68.00        32   
107        shasta lake        41       -122            79.00        52   
108        shasta lake        41       -123            69.80        43   
109             arcata        41       -124            61.00        82   

     Cloudiness  Wind Speed        Date  
0             1        4.70  2020-08-10  
1             0        4.18

In [14]:
city_data_df.to_json(r'C:\Users\rona_\Desktop\Aug10.json')